# Übung - Metadaten suchen & filtern

Somit lautet unsere neue Fragestellung also:

**Welche sind die Datensätze, die das Wort "Baumkataster" im Titel beinhalten und im Zeitabschnitt 2022-2024 erschienen sind? Welche sind die Bereitsteller, die jene Datensätze liefern? In welchen Formaten kommen die Datensätze vor?** 

Hierbei ist es wichtig zu erwähnen, dass SPARQL über keine native Paginierungsfunktion verfügt. Dies besagt, dass man leider nicht die Ergebnisse durchblättern kann. Gewöhnlicherweise wird das über die UI-Funktionalitäten umgesetzt - ein Merkmal davon, wie rudimentär SPARQL in ihrer Basis ist. Deswegen muss man in der Abfrage zusätzlich spezifizieren, welche Ergebnisse bzw. Seiten aufgelistet werden sollen.

In [1]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(CONTAINS(STR(?modified), "2022") || CONTAINS(STR(?modified), "2023") || CONTAINS(STR(?modified), "2024"))
        }
    }
}

Endpoint set to: https://data.europa.eu/sparql

In [2]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(CONTAINS(STR(?modified), "2022") || CONTAINS(STR(?modified), "2023") || CONTAINS(STR(?modified), "2024"))
        }
                LIMIT 6 OFFSET 20
    }
}

Endpoint set to: https://data.europa.eu/sparql

uri,title,contributorid,modified
http://data.europa.eu/88u/dataset/73c5a6b3-c033-4dad-bb7d-8783427dd233,Baumkataster Fráncfort del Meno,http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,2024-11-07T06:54:14.744750
http://data.europa.eu/88u/dataset/73c5a6b3-c033-4dad-bb7d-8783427dd233,Baumkataster Φραγκφούρτη,http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,2024-11-07T06:54:14.744750
http://data.europa.eu/88u/dataset/73c5a6b3-c033-4dad-bb7d-8783427dd233,Baumkataster Франкфурт на Майн,http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,2024-11-07T06:54:14.744750
http://data.europa.eu/88u/dataset/fcdceb2e-d16d-410c-ba0f-521ba8c6effa,Fachpläne - Baumkataster,http://dcat-ap.de/def/contributors/datenBW,2024-10-10T13:45:21.322762
http://data.europa.eu/88u/dataset/c1c61928-c602-4e37-af31-2d23901e2540,Straßenbaumkataster Hamburg,http://dcat-ap.de/def/contributors/transparenzportalHamburg,2024-08-09T00:10:58.572825
http://data.europa.eu/88u/dataset/19676799-fedb-4d1a-a89a-26fba887b3f0~~2,Straßenbaumkataster Hamburger Hafen,http://dcat-ap.de/def/contributors/transparenzportalHamburg,2024-08-08T03:15:01.540503


Die Abfrage mit LIMIT 6 OFFSET 20 gibt uns genau die Ergebnisse 21 bis 26 zurück. Wenn man also die Anzahl der zurückgegebenen Ergebnisse begrenzen möchte, ist es sinnvoll, LIMIT zu verwenden.

```{admonition} Erklärung des Codes
:class: tip, dropdown
Da wir uns schon mit der Struktur einer SPARQL-Abfrage auseinandergesetzt haben, können wir schon viel von der obigen ablesen. Erstmal haben wir den definierten Endpunkt, den wir nennen müssen, um auf den Standort der Metadaten hinzuweisen. Danach beschreiben wir die Prefixes, die unsere Verlinkungen in dem WHERE-Teil erleichtern. Neu hier ist der PREFIX dcatde - hier sind alle Eigentschaften verortet, die spezifisch für aus Deutschland stammende Datensätze sind. Wir verweisen somit auf contributorid - hier ist die Information über die Datenbereitsteller gespeichert. Die andere neue Eigenschaft ist modified, was besagt, wann zum letzen der jeweilige Datensatz bearbeitet worden ist. Diese Eigenschaft gibt die aktuellste Auskunft darüber, aus welchem Jahr der Datensatz stammt. Eine andere neue Bedingung, die in der Absprache zu finden ist, wäre FILTER. FILTER beschreibt eine spezifische Bedingung, die zwingend zu erfüllen ist, und somit schränkt die Ergebnisse darauf ein. Somit lassen wir uns Ergebnisse angeben, die ihre contributorid ausschließlich als URI haben und zwar die mit "http://dcat-ap.de/def/contributors/" anfangen. Das schließt alle leerstehende Ausprägungen und auch solche, die nicht in der Form von URIs sind. Wichtig sind auch die Befehle strstarts und str. Die Funktion str() konvertiert den Wert der Variable ?contributorid in einen String. In SPARQL-Abfragen werden Variablen oft als IRIs (Internationalized Resource Identifiers) dargestellt, also als URLs. Die Funktion str() nimmt diesen IRI und wandelt ihn in eine einfache Zeichenkette (String) um. Die Funktion strstarts() überprüft, ob ein gegebener String mit einer bestimmten Zeichenkette beginnt. Für die Eigenschaft modified legen wir fest, welche Jahre in der Form einer vierstelligen Zahl in der Zeichenkette sein sollten, also 2022, 2023, 2024.
```